# RUN

In [1]:
import os
import subprocess
import pickle
from tqdm import tqdm

In [2]:
command = 'cd /Users/kyoheikoyama/workspace/ligplottmp/ && python /Users/kyoheikoyama/workspace/LigPlus/command.py'
subprocess.check_call(command.split(' '),
                      shell = False)

0

# Check

In [3]:
d = '../DICT_PDB_Result/'
files = os.listdir(d)


In [4]:
for f in files:
    print(os.path.join(os.getcwd(), d,f))

/Users/kyoheikoyama/workspace/tcrpred/analysis/analysis1_alldata/../DICT_PDB_Result/DICT_PDBID_2_MELTDIST.pickle
/Users/kyoheikoyama/workspace/tcrpred/analysis/analysis1_alldata/../DICT_PDB_Result/DICT_PDBID_2_Atten12.pickle
/Users/kyoheikoyama/workspace/tcrpred/analysis/analysis1_alldata/../DICT_PDB_Result/DICT_PDBID_2_CDRS.pickle
/Users/kyoheikoyama/workspace/tcrpred/analysis/analysis1_alldata/../DICT_PDB_Result/DICT_PDBID_2_dfnnb.pickle
/Users/kyoheikoyama/workspace/tcrpred/analysis/analysis1_alldata/../DICT_PDB_Result/DICT_PDBID_2_model_out.pickle
/Users/kyoheikoyama/workspace/tcrpred/analysis/analysis1_alldata/../DICT_PDB_Result/DICT_PDBID_2_RESIDUES.pickle
/Users/kyoheikoyama/workspace/tcrpred/analysis/analysis1_alldata/../DICT_PDB_Result/DICT_PDBID_2_DISTANCE.pickle
/Users/kyoheikoyama/workspace/tcrpred/analysis/analysis1_alldata/../DICT_PDB_Result/DICT_PDBID_2_dfhhb.pickle
/Users/kyoheikoyama/workspace/tcrpred/analysis/analysis1_alldata/../DICT_PDB_Result/DICT_PDBID_2_SEQUENCES

In [5]:
import os
import subprocess
import pickle
from tqdm import tqdm


a = '/Users/kyoheikoyama/workspace/ligplottmp/'
listdirs = os.listdir(a)

for f in listdirs:
    if f=='.DS_Store':
        continue
    results = os.listdir(os.path.join(a, f))
    if len(results)==9:
        continue
    else:
        print(f)

In [6]:
len(listdirs)

78

In [7]:
!ls /Users/kyoheikoyama/workspace/tcrpred/analysis/analysis1_alldata/pdb/ | wc

     895     895   10740


In [8]:
# r_0 = DICT_PDB_Result['DICT_PDBID_2_CDRS.pickle']['2VLK'][-1][0]
# r_m1 = DICT_PDB_Result['DICT_PDBID_2_CDRS.pickle']['2VLK'][-1][-1]
# r_0.get_full_id()[-1][1], r_m1.get_full_id()[-1][1]

In [16]:
# %load /Users/kyoheikoyama/workspace/LigPlus/command.py
import subprocess
import os, pickle
from tqdm import tqdm

"""
"LigPlot command line" needs ResidueName and LigandPosition.

- Command mannual
https://www.ebi.ac.uk/thornton-srv/software/LIGPLOT/manual/
https://www.ebi.ac.uk/thornton-srv/software/LIGPLOT/manual/man2.html

- Output format
UI mannual
https://www.ebi.ac.uk/thornton-srv/software/LigPlus/manual2/manual.html
"""


### Define LigPlot+ environment here
components_cif = '/Users/kyoheikoyama/workspace/LigPlus/lib/params/components.cif' # Location of components.cif
ligplot_plus = '/Users/kyoheikoyama/workspace/LigPlus/lib/exe_mac/' # Location of your LigPlus executable folder
pdbdir = '/Users/kyoheikoyama/workspace/tcrpred/analysis/analysis1_alldata/pdb/'
path_DICT_PDBID_2_CDRS = "/Users/kyoheikoyama/workspace/tcrpred/analysis/analysis1_alldata/../DICT_PDB_Result/DICT_PDBID_2_CDRS.pickle"
path_DICT_PDBID_2_CHAINNAMES = "/Users/kyoheikoyama/workspace/tcrpred/analysis/analysis1_alldata/../DICT_PDB_Result/DICT_PDBID_2_CHAINNAMES.pickle"

LIGPLOT_ROOT = "/Users/kyoheikoyama/workspace/ligplottmp"

os.system(f"mkdir {LIGPLOT_ROOT}/hbadd_hbplus_result/")


def run_hbplus(filename):
	subprocess.check_call(['{}hbadd'.format(ligplot_plus), filename, components_cif, '-wkdir', f'{LIGPLOT_ROOT}/hbadd_hbplus_result/'], shell = False)
	file_prefix = filename.split('/')[-1].replace('.ent', '')

	if len(file_prefix)!=7:
		print(file_prefix, filename)
		assert False

	subprocess.check_call(['{}hbplus'.format(ligplot_plus), '-L', '-h', '2.90', '-d', '3.90', '-N', filename, '-wkdir', f'{LIGPLOT_ROOT}/hbadd_hbplus_result/'], shell = False)
	subprocess.check_call(['{}hbplus'.format(ligplot_plus), '-L', '-h', '2.70', '-d', '3.35', filename, '-wkdir', f'{LIGPLOT_ROOT}/hbadd_hbplus_result/'], shell = False)



def run_all(filename, res1, res2, ligand_chain):
	"""Emulates running the LigPlot+ DIMPLOT algorithm. Rewriting as a CLI to allow for a batch mode."""

	file_prefix = filename.split('/')[-1].replace('.ent', '')
	if len(file_prefix)!=7:
		print(file_prefix, filename)
		assert False

	assert len(file_prefix)==7

	if os.path.exists(file_prefix):
		return

	# Run HBadd
	subprocess.check_call(['{}hbadd'.format(ligplot_plus), filename, components_cif, '-wkdir', './'], shell = False)


	# Run HBplus
	subprocess.check_call(['{}hbplus'.format(ligplot_plus), '-L', '-h', '2.90', '-d', '3.90', '-N', filename, '-wkdir', './'], shell = False)


	# Run HBplus again
	subprocess.check_call(['{}hbplus'.format(ligplot_plus), '-L', '-h', '2.70', '-d', '3.35', filename, '-wkdir', './'], shell = False)


	# Run ligplot
	# ligplot filename [residue1] [residue2] [chain_id] [-w] [-m].
	# [residue1] and [residue2] is the residue range for the ligand
	subprocess.check_call([
		'{}ligplot'.format(ligplot_plus), filename, str(res1), str(res2), ligand_chain, '-wkdir', './',
		'-prm', '/Users/kyoheikoyama/workspace/LigPlus/lib/params/ligplot.prm'],
		shell = False)

	files_to_rename = [_ for _ in os.listdir('.') if 'dimplot.' in _[0:8] or 'ligplot.' in _[0:8]]

	os.system(f'mkdir {file_prefix}')

	for file_to_rename in files_to_rename:
		subprocess.check_call([
			'mv', file_to_rename, f'./{file_prefix}/{file_to_rename}'])

def main():
	"""Main function."""
	# Get list of pdb files in the directory
	DICT_PDBID_2_CDRS = pickle.load(open(path_DICT_PDBID_2_CDRS, "rb"))
	DICT_PDBID_2_CHAINNAMES = pickle.load(open(path_DICT_PDBID_2_CHAINNAMES, "rb"))
	# pickleload = lambda p: pickle.load(open(p,"rb"))
	# DICT_PDBID_2_Atten12 = pickleload("/Users/kyoheikoyama/workspace/tcrpred/analysis/analysis1_alldata/../DICT_PDB_Result/DICT_PDBID_2_Atten12.pickle")

	for pdbid in tqdm(list(DICT_PDBID_2_CHAINNAMES.keys())):
		alpha_chain, beta_chain, ligand_chain = DICT_PDBID_2_CHAINNAMES[pdbid]
		pdb_file = os.path.join(pdbdir, f"pdb{pdbid.lower()}.ent")
		r_0 = DICT_PDBID_2_CDRS[pdbid][-1][0]
		r_m1 = DICT_PDBID_2_CDRS[pdbid][-1][-1]
		res1, res2 = r_0.get_full_id()[-1][1], r_m1.get_full_id()[-1][1]

		print("*"*400)
		print("""pdb_file, res1, res2, ligand_chain""")
		print(pdb_file, res1, res2, ligand_chain)
		try:
			run_hbplus(pdb_file, res1, res2, ligand_chain)
			run_all(pdb_file, res1, res2, ligand_chain)
		except:
			continue
	# quit()
	print('done!')

def hbplus():
	DICT_PDBID_2_CDRS = pickle.load(open(path_DICT_PDBID_2_CDRS, "rb"))
	DICT_PDBID_2_CHAINNAMES = pickle.load(open(path_DICT_PDBID_2_CHAINNAMES, "rb"))

	for pdbid in tqdm(list(DICT_PDBID_2_CHAINNAMES.keys())):
		alpha_chain, beta_chain, ligand_chain = DICT_PDBID_2_CHAINNAMES[pdbid]
		pdb_file = os.path.join(pdbdir, f"pdb{pdbid.lower()}.ent")
		r_0 = DICT_PDBID_2_CDRS[pdbid][-1][0]
		r_m1 = DICT_PDBID_2_CDRS[pdbid][-1][-1]
		res1, res2 = r_0.get_full_id()[-1][1], r_m1.get_full_id()[-1][1]

		print("*"*400)
		print("""pdb_file, res1, res2, ligand_chain""")
		print(pdb_file, res1, res2, ligand_chain)
		try:
			run_hbplus(pdb_file, res1, res2, ligand_chain)
		except:
			continue
	# quit()
	print('done!')
    
    
# if __name__ == '__main__':
# 	try:
# 		main()
# 	except KeyboardInterrupt:
# 		print('\n\nGoodbye!\n\n')



In [ ]:
DICT_PDBID_2_CHAINNAMES = DICT_PDBID_2_CHAINNAMES = pickle.load(open(path_DICT_PDBID_2_CHAINNAMES, "rb"))
for pdbid in tqdm(list(DICT_PDBID_2_CHAINNAMES.keys())):
    pdb_file = os.path.join(pdbdir, f"pdb{pdbid.lower()}.ent")
    run_hbplus(pdb_file)

 86%|████████▌ | 68/79 [04:10<00:36,  3.31s/it]

# Result

In [16]:
!ls /Users/kyoheikoyama/workspace/ligplottmp/pdb4ozi/

ligplot.bonds ligplot.frm   ligplot.nnb   ligplot.ps    ligplot.sum
ligplot.drw   ligplot.hhb   ligplot.pdb   ligplot.rcm


In [19]:
!cat /Users/kyoheikoyama/workspace/ligplottmp/pdb5jzi/ligplot.frm | head

ATOM     90  N   TYR A   7     -88.372  80.096  30.189  1.00 38.19           N
ATOM     91  CA  TYR A   7     -87.301  80.202  29.216  1.00 37.69           C
ATOM     92  C   TYR A   7     -87.786  81.051  28.062  1.00 37.22           C
ATOM     93  O   TYR A   7     -88.902  80.866  27.590  1.00 37.13           O
ATOM     94  CB  TYR A   7     -86.879  78.828  28.712  1.00 38.01           C
ATOM     95  CG  TYR A   7     -86.159  78.007  29.746  1.00 39.39           C
ATOM     96  CD1 TYR A   7     -86.798  76.958  30.392  1.00 39.61           C
ATOM     97  CD2 TYR A   7     -84.845  78.280  30.082  1.00 40.90           C
ATOM     98  CE1 TYR A   7     -86.146  76.202  31.341  1.00 41.77           C
ATOM     99  CE2 TYR A   7     -84.183  77.530  31.031  1.00 43.23           C


In [20]:
!cat /Users/kyoheikoyama/workspace/ligplottmp/pdb5jzi/ligplot.bonds | head

Bond   0. Atoms16495 [ CA  GLY   96  E] -> 5823 [ OD1 ASN    8  C]n E 3.6296 - HBPLUS
Bond   1. Atoms13362 [ OD2 ASP  102  D] -> 5802 [ CB  ILE    7  C]n E 3.6168 - HBPLUS
Bond   2. Atoms13360 [ CG  ASP  102  D] -> 5801 [ O   ILE    7  C]n E 3.6551 - HBPLUS
Bond   3. Atoms13362 [ OD2 ASP  102  D] -> 5799 [ CA  ILE    7  C]n E 3.7304 - HBPLUS
Bond   4. Atoms13360 [ CG  ASP  102  D] -> 5798 [ N   ILE    7  C]n E 3.7411 - HBPLUS
Bond   5. Atoms13362 [ OD2 ASP  102  D] -> 5793 [ C   GLY    6  C]n E 3.8662 - HBPLUS
Bond   6. Atoms13360 [ CG  ASP  102  D] -> 5791 [ N   GLY    6  C]n E 3.4774 - HBPLUS
Bond   7. Atoms16506 [ CG  PRO   97  E] -> 5779 [ CD2 LEU    5  C]n E 3.6240 - HBPLUS
Bond   8. Atoms16505 [ CB  PRO   97  E] -> 5779 [ CD2 LEU    5  C]n E 3.8296 - HBPLUS
Bond   9. Atoms12339 [ OH  TYR   38  D] -> 5779 [ CD2 LEU    5  C]n E 3.8371 - HBPLUS


In [21]:
!cat /Users/kyoheikoyama/workspace/ligplottmp/pdb5jzi/ligplot.nnb | head

ligplot.nnb output:

   Atom 1               Atom 2        Distance
GLY E   96   CA      ASN C    8   OD1    3.63
ASP D  102   OD2     ILE C    7   CB     3.62
ASP D  102   CG      ILE C    7   O      3.66
ASP D  102   OD2     ILE C    7   CA     3.73
ASP D  102   CG      ILE C    7   N      3.74
ASP D  102   OD2     GLY C    6   C      3.87
ASP D  102   CG      GLY C    6   N      3.48


In [22]:
!cat /Users/kyoheikoyama/workspace/ligplottmp/pdb5jzi/ligplot.hhb | head

ligplot.hhb output:

   Donor                  Acceptor     Distance
ILE C    7   N       ASP D  102   OD2    3.00
ASN C    8   ND2     GLY C    6   O      2.62
GLY C    6   N       ASP D  102   OD2    2.96
GLY C    6   N       ASP D  102   OD1    3.23
ASP D  102   N       ALA C    4   O      3.23
LYS C    1   NZ      SER D   33   O      2.43
LYS C    1   N       TYR A  171   OH     3.18
